In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE230164"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE230164"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE230164.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE230164.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE230164.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

### Step 2: Dataset Analysis and Clinical Feature Extraction

### Step 3: Gene Data Extraction

### Step 4: Gene Identifier Review

In [ ]:
# First, let's load the gene expression data to examine the gene identifiers
try:
    # Load gene expression data from a previous step
    gene_file = os.path.join(in_cohort_dir, 'gene_expression.txt')
    gene_data = pd.read_csv(gene_file, sep='\t', index_col=0)
    
    # Look at the first few gene identifiers
    gene_identifiers = gene_data.index.tolist()[:10]  # Sample of gene identifiers
    print("Sample gene identifiers:", gene_identifiers)
    
    # Check if identifiers are likely human gene symbols
    # Human gene symbols typically have format like "BRCA1", "TP53", etc.
    # Other formats might be Ensembl IDs (ENSG...), Affymetrix IDs, or probe IDs
    
    # Simple heuristic: If most identifiers match pattern of human gene symbols
    # (typically uppercase letters with some numbers, not starting with numbers)
    import re
    
    gene_symbol_pattern = re.compile(r'^[A-Z][A-Z0-9]*$')
    ensembl_pattern = re.compile(r'^ENS[A-Z]*[0-9]+')
    probe_pattern = re.compile(r'^[0-9]+_')
    
    gene_symbol_count = sum(1 for gene_id in gene_identifiers if gene_symbol_pattern.match(gene_id))
    ensembl_count = sum(1 for gene_id in gene_identifiers if ensembl_pattern.match(gene_id))
    probe_count = sum(1 for gene_id in gene_identifiers if probe_pattern.match(gene_id))
    
    print(f"Gene symbol pattern matches: {gene_symbol_count}/{len(gene_identifiers)}")
    print(f"Ensembl pattern matches: {ensembl_count}/{len(gene_identifiers)}")
    print(f"Probe pattern matches: {probe_count}/{len(gene_identifiers)}")
    
    # Determine if mapping is needed
    requires_gene_mapping = not (gene_symbol_count / len(gene_identifiers) > 0.8)
    print(f"requires_gene_mapping = {requires_gene_mapping}")
    
except Exception as e:
    print(f"Error examining gene identifiers: {e}")
    # If we can't determine, default to requiring mapping
    requires_gene_mapping = True
    print(f"requires_gene_mapping = {requires_gene_mapping}")


### Step 5: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Let's identify the relevant columns for mapping
# Based on the preview, 'ID' seems to be the probe identifier, and 'Symbol' contains gene symbols
# These appear to match what we need for mapping

# 2. Get gene expression data
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_data.shape}")
print("First few probe IDs from gene expression data:")
print(gene_data.index[:5])

# 3. Get gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 4. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"After mapping: Gene expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# 5. Normalize gene symbols using synonym information
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: Gene expression data shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# 6. Save gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. The gene data is already normalized in the previous step

# 2. Link the clinical and genetic data
print("Extracting clinical data directly from the source...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get unique values dictionary to identify feature rows
unique_values_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics (first few entries):")
for i, (key, values) in enumerate(unique_values_dict.items()):
    print(f"{key}: {values}")
    if i > 5:  # Limit output to first few entries
        print("...")
        break

# Define conversion functions based on the data inspection
# These would normally be defined in Step 2
def convert_trait(cell):
    """Convert allergies information to binary (1: has allergies, 0: healthy control)"""
    if isinstance(cell, str):
        if 'allergy' in cell.lower() or 'allergic' in cell.lower():
            return 1
        elif 'healthy' in cell.lower() or 'control' in cell.lower():
            return 0
    return None

def convert_age(cell):
    """Extract age value from cell"""
    if isinstance(cell, str) and 'age:' in cell.lower():
        # Extract numbers after "age:"
        import re
        match = re.search(r'age:\s*(\d+)', cell.lower())
        if match:
            return float(match.group(1))
    return None

def convert_gender(cell):
    """Convert gender to binary (0: female, 1: male)"""
    if isinstance(cell, str):
        cell = cell.lower()
        if 'female' in cell or 'f' in cell:
            return 0
        elif 'male' in cell or 'm' in cell:
            return 1
    return None

# Find appropriate rows for trait, age, and gender
trait_row = None
age_row = None
gender_row = None

# Scan through the unique values to identify feature rows
for idx, values in unique_values_dict.items():
    values_str = str(values).lower()
    if 'allergy' in values_str or 'allergic' in values_str or 'healthy' in values_str:
        trait_row = idx
    elif 'age' in values_str:
        age_row = idx
    elif 'gender' in values_str or 'sex' in values_str or ('male' in values_str and 'female' in values_str):
        gender_row = idx

print(f"Identified trait_row: {trait_row}, age_row: {age_row}, gender_row: {gender_row}")

# Extract clinical features
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    
    # 3. Handle missing values in the linked data
    print("Handling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Check if trait is biased
    print("Checking for bias in trait distribution...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Final validation
    note = "Dataset contains gene expression data from peripheral blood related to food allergies."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,  # We have gene data
        is_trait_available=True,  # We've identified the trait row
        is_biased=is_biased,
        df=linked_data,
        note=note
    )
    
    print(f"Dataset usability: {is_usable}")
    
    # 6. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for trait-gene association studies due to bias or other issues.")
else:
    print("No trait information found in the clinical data. Cannot proceed with linking.")
    # Save validation information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=None,
        note="No trait information found in the clinical data."
    )
    print(f"Dataset usability: {is_usable}")